In [47]:
from langgraph.graph import StateGraph,START,END
from typing import TypedDict,Literal ,Annotated 
from langchain_core.messages import SystemMessage,AIMessage,HumanMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import operator

In [48]:
load_dotenv()
model= ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [49]:
generator_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
evaluator_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
optimiser_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [50]:
from pydantic import BaseModel, Field

class TweetEvaluation(BaseModel):
    evaluation: Literal["approved", "needs_improvement"] = Field(..., description="Final evaluation result.")
    feedback: str = Field(..., description="feedback for the tweet.")

    
structured_evaluator_llm = evaluator_llm.with_structured_output(TweetEvaluation)

In [51]:
# state

class XTwitState(TypedDict):
    topic:str
    tweet:str
    evaluation: Literal['approved','needs_improvement']
    feedback:str
    iteration:int
    max_iteration:int


    tweet_History: Annotated[list[str],operator.add]
    feedback_History: Annotated[list[str],operator.add]

In [52]:
def generate(state:XTwitState):
    topic = state['topic']
    #prompt
    messages = [
        SystemMessage(content="You are a funny and clever Twitter/X influencer."),
        HumanMessage(content=f"""
Write a short, original, and hilarious tweet on the topic: "{state['topic']}".

Rules:
- Do NOT use question-answer format.
- Max 280 characters.
- Use observational humor, irony, sarcasm, or cultural references.
- Think in meme logic, punchlines, or relatable takes.
- Use simple, day to day english
""")

# -This is version {state['iteratoin']+1}.
    ]
    #send to generate tweet
    response = generator_llm.invoke(messages).content

    # return response
    return {'tweet':response,'tweet_History':[response]}

In [53]:
def evaluate(state:XTwitState):
    #prompt
    messages = [
    SystemMessage(content="You are a ruthless, no-laugh-given Twitter critic. You evaluate tweets based on humor, originality, virality, and tweet format."),
    HumanMessage(content=f"""
Evaluate the following tweet:

Tweet: "{state['tweet']}"

Use the criteria below to evaluate the tweet:

1. Originality – Is this fresh, or have you seen it a hundred times before?  
2. Humor – Did it genuinely make you smile, laugh, or chuckle?  
3. Punchiness – Is it short, sharp, and scroll-stopping?  
4. Virality Potential – Would people retweet or share it?  
5. Format – Is it a well-formed tweet (not a setup-punchline joke, not a Q&A joke, and under 280 characters)?

Auto-reject if:
- It's written in question-answer format (e.g., "Why did..." or "What happens when...")
- It exceeds 280 characters
- It reads like a traditional setup-punchline joke
- Dont end with generic, throwaway, or deflating lines that weaken the humor (e.g., “Masterpieces of the auntie-uncle universe” or vague summaries)

### Respond ONLY in structured format:
- evaluation: "approved" or "needs_improvement"  
- feedback: One paragraph explaining the strengths and weaknesses 
""")
]


    #evaluuate result

    response = structured_evaluator_llm.invoke(messages)

    #response 
    return{'evaluation':response.evaluation,'feedback':response.feedback , 'feedback_history':[response.feedback]}

In [54]:
def optimize(state:XTwitState):
    #propmt
    messages = [
        SystemMessage(content="You punch up tweets for virality and humor based on given feedback."),
        HumanMessage(content=f"""
Improve the tweet based on this feedback:
"{state['feedback']}"

Topic: "{state['topic']}"
Original Tweet:
{state['tweet']}

Re-write it as a short, viral-worthy tweet. Avoid Q&A style and stay under 280 characters.
""")
    ]

    response =  optimiser_llm.invoke(messages).content 
    iteration = state['iteration'] + 1
    return{'tweet':response , 'iteration':iteration , 'tweet_History':[response]}

In [55]:
def check_conditions(state:XTwitState)->Literal['approved','needs_improvement']:
    if state['evaluation'] == "approved" or state['iteration']>= state['max_iteration']:
        return "approved"
    else:
        return 'needs_improvement'

In [56]:
graph =StateGraph(XTwitState)

# add node

graph.add_node("generate",generate)
graph.add_node("evaluate",evaluate)
graph.add_node("optimize",optimize)



# add edges



graph.add_edge(START,"generate")
graph.add_edge("generate","evaluate")


graph.add_conditional_edges('evaluate',check_conditions,{'approved':END,"needs_improvement":"optimize"})


graph.add_edge('optimize','evaluate')

workflow = graph.compile()

In [68]:
initial_state = {
    "topic": "ffjwolywnwrsgdksagdilasiudgasdgasidulgasivd ayuidyboqwi8yoiyoai8dyi8qwyo8e12ye ewqidilwqydilyqwiydowyildo",
    "iteration": 1,
    "max_iteration": 5
}
result = workflow.invoke(initial_state)

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 22
}
].
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing de

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-flash"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 50
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 45
}
]

In [69]:
result

{'topic': 'ffjwolywnwr',
 'tweet': 'My autocorrect just changed "ffjwolywnwr" to "fluffy kitten mittens."  My life is officially a rom-com.  Send wine and existential dread. #autocorrectfail #mylifeisamovie #sendhelp',
 'evaluation': 'approved',
 'feedback': 'The tweet is original and humorous, with a good virality potential. The length is appropriate, and the format is well-formed. The autocorrect mishap is relatable, and the witty response ("send wine and existential dread") elevates the humor.  The hashtags are relevant and will likely improve visibility.',
 'iteration': 1,
 'max_iteration': 5,
 'tweet_History': ['My autocorrect just changed "ffjwolywnwr" to "fluffy kitten mittens."  My life is officially a rom-com.  Send wine and existential dread. #autocorrectfail #mylifeisamovie #sendhelp'],
 'feedback_History': []}